## Linear Regression - Homework

__New Your City Airbnb price prediction__  
Link to the [homework](https://github.com/alexeygrigorev/mlbookcamp-code/blob/master/course-zoomcamp/02-regression/homework.md)

Importing the libraries:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### 1. Getting and cleaning the data

Getting the dataset:

In [5]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AB_NYC_2019.csv'

In [6]:
!wget $data

--2021-12-27 16:21:17--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AB_NYC_2019.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7077973 (6,8M) [text/plain]
Saving to: ‘AB_NYC_2019.csv’

AB_NYC_2019.csv     100%[===================>]   6,75M  4,58MB/s    in 1,5s    

2021-12-27 16:21:19 (4,58 MB/s) - ‘AB_NYC_2019.csv’ saved [7077973/7077973]

